# combine_data_sources.ipynb
**Combines the different data sources to create one dataframe containing all data (EV registrations, EV product variety, gas prices, EV charging locations, ...) by month and ZIP code and saves the resulting dataframe in `data/df.csv`.**

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
from uszipcode import SearchEngine
search = SearchEngine()

import utils as u
from config.counties_zips import zip_codes

In [3]:
times = [u.get_last_day_of_month(dt.datetime(year=y, month=m, day=20)) for y in range(2017,2022) for m in range(1,13)]
# times = [u.get_last_day_of_month(dt.datetime(year=y, month=m, day=20)) for y in range(2017,2018) for m in range(1,3)]
# zip_codes = df["zip"].unique()

In [4]:
multiindex = pd.MultiIndex.from_product([times, zip_codes], names=["time", "zip"])

In [5]:
# create final dataframe and add county column to it
df = pd.DataFrame(index=multiindex, columns=["county"])
for zip_code in zip_codes:
    df.loc[(slice(None), zip_code), "county"] = search.by_zipcode(zip_code).county.replace(" County", "")
df

county
time       zip               
2017-01-31 98001         King
           98002         King
           98003         King
           98004         King
           98005         King
...                       ...
2021-12-31 99363  Walla Walla
           99371        Adams
           99401       Asotin
           99402       Asotin
           99403       Asotin

[35880 rows x 1 columns]

In [8]:
# df.to_csv("data/index.csv")

In [9]:
# gas_price
df_gas = pd.read_csv("data/gas/Weekly_Retail_Gasoline_and_Diesel_Prices.csv", header=6)#, parse_dates=["Month"])
df_gas = df_gas.rename(columns={"Month" : "time"})

df_gas["time"] = pd.to_datetime(df_gas["time"])
df_gas["time"] = df_gas["time"].apply(u.get_last_day_of_month)
df_gas = df_gas.set_index("time")

for time in times:
    gas_price = df_gas.loc[time, "Washington All Grades All Formulations Retail Gasoline Prices $/gal"]#.values[0]
    df.loc[(time, slice(None)), "gas_price"] = gas_price
df.head()

county  gas_price
time       zip                    
2017-01-31 98001   King      2.743
           98002   King      2.743
           98003   King      2.743
           98004   King      2.743
           98005   King      2.743

In [10]:
df.shape

(35880, 2)

In [11]:
# n_ev (number of registered EVs), m_ev (number of different EV make/models ["product variety"])
df_ev = pd.read_csv("data/vehicles/ev_counts.csv")
df_ev["time"] = pd.to_datetime(df_ev["time"]).dt.date
df_ev = df_ev.set_index(["time", "zip"])

##### df = df.merge(df_ev, on=["time", "zip"], how="inner", suffixes=(None,"_y"))
df = df.join(df_ev, on=["time", "zip"], how="inner", rsuffix="_y")
df = df.drop(columns="county_y") #the county column appears twice, so removing the duplicate he

# alternative way (instead of using join):
# for time in times:
#     print(time)
#     for zip_code in zip_codes:
#         ev_counts = df_ev.loc[(time, zip_code), cols]
#         for col in cols:
#             if not np.isnan(ev_counts[col]):
#                 df.loc[(time, zip_code), col] = ev_counts[col]

print("done")

done


In [12]:
df.head()

county  gas_price   n_ev  n_bev  n_phev  m_ev  m_bev  m_phev
time       zip                                                               
2017-01-31 98001   King      2.743   73.0   34.0    39.0    43     24      20
           98002   King      2.743   32.0   17.0    15.0    43     24      20
           98003   King      2.743   54.0   22.0    32.0    43     24      20
           98004   King      2.743  397.0  306.0    91.0    43     24      20
           98005   King      2.743  194.0  138.0    56.0    43     24      20

In [13]:
df.shape

(35880, 8)

In [19]:
#evse
#number of installed L2 and DCFC station locations by zip code and month
df_evse = pd.read_csv("data/evse/EV_charging_stations_WA.csv")
df_evse_key = pd.read_csv("config/EV_charging_stations_key.csv")
df_evse.columns = df_evse_key["name"]

df_evse["open_date"] = pd.to_datetime(df_evse["open_date"]).dt.date

df_evse["l2_count"] = df_evse["l2_count"].fillna(0).astype(int)
df_evse["dcfc_count"] = df_evse["dcfc_count"].fillna(0).astype(int)

# df_evse["has_l2_or_dcfc"] = df_evse["l2_count"] > 0 #boolean if location has at least one L2 charger
df_evse["has_l2_or_dcfc"] = (df_evse["l2_count"] > 0) | (df_evse["dcfc_count"] > 0) #boolean if location has at least one L2 or DCFC charger

df["n_evse"] = 0
for time in times:
    print(time, "", end="")
    for zip_code in zip_codes:
        dff = df_evse[(df_evse["open_date"] < time) & (df_evse["zip"] == zip_code)] #create helper dataframe containing all stations built in this zip code until this time
        n_evse = sum(dff["has_l2_or_dcfc"]) #counts number of locations with at least one charging station in that ZIP code and for that month
        df.loc[(time, zip_code), "n_evse"] = n_evse
print("done")

2017-01-31 2017-02-28 2017-03-31 2017-04-30 2017-05-31 2017-06-30 2017-07-31 2017-08-31 2017-09-30 2017-10-31 2017-11-30 2017-12-31 2018-01-31 2018-02-28 2018-03-31 2018-04-30 2018-05-31 2018-06-30 2018-07-31 2018-08-31 2018-09-30 2018-10-31 2018-11-30 2018-12-31 2019-01-31 2019-02-28 2019-03-31 2019-04-30 2019-05-31 2019-06-30 2019-07-31 2019-08-31 2019-09-30 2019-10-31 2019-11-30 2019-12-31 2020-01-31 2020-02-29 2020-03-31 2020-04-30 2020-05-31 2020-06-30 2020-07-31 2020-08-31 2020-09-30 2020-10-31 2020-11-30 2020-12-31 2021-01-31 2021-02-28 2021-03-31 2021-04-30 2021-05-31 2021-06-30 2021-07-31 2021-08-31 2021-09-30 2021-10-31 2021-11-30 2021-12-31 done


In [20]:
df.head()

county  gas_price   n_ev  n_bev  n_phev  m_ev  m_bev  m_phev  \
time       zip                                                                  
2017-01-31 98001   King      2.743   73.0   34.0    39.0    43     24      20   
           98002   King      2.743   32.0   17.0    15.0    43     24      20   
           98003   King      2.743   54.0   22.0    32.0    43     24      20   
           98004   King      2.743  397.0  306.0    91.0    43     24      20   
           98005   King      2.743  194.0  138.0    56.0    43     24      20   

                  n_evse  
time       zip            
2017-01-31 98001       0  
           98002       2  
           98003       1  
           98004      16  
           98005       1

In [21]:
df.shape

(35880, 9)

In [ ]:
#save resulting dataframe as csv file
df.to_csv("data/df.csv")